# Name- Anuj Chandak
Msc Statistics

The below code is the pipeline which was to be build as mentioned in the screening test document

In [8]:
#Here we are firstly importing the libraries which are required for the given model.
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
#libraries for ML models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor,ExtraTreesClassifier 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import ElasticNet
from sklearn import svm
from sklearn import neural_network

In [9]:
# 1)parse the JSON file and Read the target and type of regression to be run
def parse_json_file(json_file):
    with open(json_file, 'r') as f:
        json_data = json.load(f)
    dataset_name = json_data['dataset_name']
    target_variable = json_data['target_variable']
    prediction_type = json_data['prediction_type']
    feature_handling = json_data['feature_handling']
    feature_reduction_method = json_data['feature_reduction_method']
    selected_algorithms = json_data['selected_algorithms']
    hyperparameters = json_data['hyperparameters']
    return dataset_name, target_variable,prediction_type,feature_handling,feature_reduction_method, selected_algorithms,hyperparameters

In [10]:
#2) Read the features and figure out what missing imputation needs to be applied and apply that to the columns loaded in a datafrme
def Load_and_data_preprocessing(csv_file):
 # Load dataset
    data = pd.read_csv(csv_file)
 # Handle missing values if any
    data.dropna(inplace=True)
 # Encode categorical variables if any
 # Split data into features and target variable
    X = data.drop(columns=[target_variable])
    y = data[target_variable]
 # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
 # Perform feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

In [11]:
# 3) Compute feature reduction based on input. 
#    There can be No Reduction, 
#    Corr with Target, Tree-based,PCA.
def feature_reduction(X_train, X_test, y_train, feature_reduction_method):
    if feature_reduction_method == 'tree_based':
 # Use RandomForestRegressor for feature selection
        selector = SelectFromModel(RandomForestRegressor())
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)
        return X_train_selected, X_test_selected
    else:
        return X_train, X_test  # No feature reduction

In [12]:
# 4)Parse the Json and make the model objects 
def model_creation(selected_algorithms):
    models = []
    for algorithm in selected_algorithms:
        if algorithm == 'RandomForestClassifier':
            models.append(('Random Forest Classifier', RandomForestClassifier()))
        elif algorithm == 'RandomForestRegressor':
            models.append(('Random Forest Regressor', RandomForestRegressor()))  
        
        elif algorithm == 'GradientBoostingClassifier':
            models.append(('Gradient Boosting Classifier', GradientBoostingC()))
        elif algorithm == 'GradientBoostingRegressor':
            models.append(('Gradient Boosting Regressor', GradientBoostingRegressor()))
       
        elif algorithm == 'LinearRegression':
            models.append(('Linear Regression', LinearRegression()))
        elif algorithm == 'LogisticRegression':
            models.append(('Logistic Regression', LogisticRegression()))
        
        elif algorithm == 'RidgeRegression':
            models.append(('RidgeRegression',Ridge()))
        elif algorithm == 'LassoRegression':
            models.append(('LassoRegression',Lasso()))
        elif algorithm == 'ElasticNetRegression':
            models.append(('ElasticNetRegression',ElasticNet()))
        
        elif algorithm == 'XGBClassifier':
            models.append(('XGBoost Classifier', XGBClassifier()))
        elif algorithm == 'XGBRegressor':
            models.append(('XGBoost Regressor', XGBRegressor()))
    
            
        elif algorithm == 'DecisionTreeRegressor':
            models.append(('Decision Tree Regressor', DecisionTreeRegressor()))
        elif algorithm == 'DecisionTreeClassifier':
            models.append(('Decision Tree Classifier', DecisionTreeClassifier()))
        
        elif algorithm == 'Support Vector Machine':
            models.append(('Support Vector Machine',svm()))
        
        elif algorithm == 'SGDRegressor':
            models.append(('Stochastic Gradient Descent Regressor', SGDRegressor()))
        elif algorithm == 'SGDClassifier':
            models.append(('Stochastic Gradient Descent Classifier', SGDClassifier()))
    
        elif algorithm == 'KNeighborsRegressor':
            models.append(('K Neighbors Regressor', KNeighborsRegressor()))
        elif algorithm == 'KNeighborsClassifier':
            models.append(('K Neighbors Classifier', KNeighborsClassifier()))
  
        elif algorithm == 'ExtraTreesRegressor':
            models.append(('Extra Trees Regressor', ExtraTreesRegressor()))
        elif algorithm == 'ExtraTreesClassifier':
            models.append(('Extra Trees Classifier', ExtraTreesClassifier()))
        
        elif algorithm == 'MLPRegressor':
            models.append(('Multi-layer Perceptron Regressor', MLPRegressor()))
        elif algorithm == 'MLPClassifier':
            models.append(('Multi-layer Perceptron Classifier', MLPClassifier()))
        
    return models


In [13]:
#5) Run the fit and predict on each model – keep in mind that you need to do hyper parameter tuning i.e., use GridSearchCV
def hyperparameter(models, hyperparameters, X_train, y_train):
    tuned_models = []
    for name, model in models:
        if name in hyperparameters:
            param_grid = hyperparameters[name]
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='negative')
            grid_search.fit(X_train, y_train)
            tuned_models.append((name, grid_search.best_estimator_))
        else:
            tuned_models.append((name, model))
    return tuned_models

In [14]:
# 6) Log to the console the standard model metrics that apply
def evaluate_models(models, X_test, y_test):
    output = {}
    for name, model in models:
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        output[name] = {'R-squared': r2,'Mean Squared Error': mse}
    return output